In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer 
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold 

In [ ]:
nltk.download('stopwords')

In [3]:
#Carregando o dataset COVID19BR

dados = pd.read_csv('covidbr_labeled.csv')
X = dados['text']
y = dados['misinformation']
dados.head()

,shares,text,misinformation,source,revision
0,27,"O ministro da Ciência, Tecnologia, Inovações e...",0,https://www.gov.br/pt-br/noticias/educacao-e-p...,NaN
1,26,Pesquisa com mais de 6.000 médicos em 30 paíse...,1,https://www.aosfatos.org/noticias/e-falso-que-...,NaN
2,25,É com muita alegria que comunico que mais um p...,0,http://portal.mec.gov.br/component/content/art...,NaN
3,25,Renda Brasil unificará vários programas sociai...,0,https://agenciabrasil.ebc.com.br/politica/noti...,NaN
4,24,O Secretário-Geral da OTAN Jens Stoltenberg ta...,0,NaN,1.0


In [4]:
stop_por = set(stopwords.words("portuguese"))
def limpa_texto(string):
    text = string.lower().split()
    text = " ".join(text)
    text = re.sub(r"http(\S)+",' ',text)    
    text = re.sub(r"www(\S)+",' ',text)
    text = re.sub(r"&",' and ',text)  
    text = re.sub('&amp',' ', text)
    text = re.sub(" \d+", " ", text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.split()
    text = [w for w in text if not w in stop_por]
    text = " ".join(text)
    return text

In [5]:
X = X.map(lambda x: limpa_texto(x))

In [6]:
# CountVectorizer / TfidfTransformer Pode ser resumido nisto: 
tfidf_vectorizer=TfidfVectorizer(use_idf=True)
#basta enviar todos os seus documentos aqui
fitted_vectorizer=tfidf_vectorizer.fit(X)
tfidf_vectorizer_vectors=fitted_vectorizer.transform(X)

# Testes com dataset COVID19BR - LinearSVC

In [7]:
X_ = tfidf_vectorizer_vectors

#Implementando a validação cruzada
 
k = 5
kf = KFold(n_splits=k, random_state=None)
model = LinearSVC()

#lista das matrizes de confusão de cada iteração 
lista_conf_matrix = []
lista_acc_score = []
lista_precision_score = []
lista_recall_score = []
lista_f1_score = []
 
for train_index , test_index in kf.split(X):
    X_train , X_test = X_[train_index],X_[test_index]
    y_train , y_test = y[train_index],y[test_index]
     
    model.fit(X_train,y_train)
    pred_values = model.predict(X_test)
     
    #print(confusion_matrix(y_test, pred_values))
    conf_matrix = confusion_matrix(y_test, pred_values)
    lista_conf_matrix.append(conf_matrix)
    
    acc = accuracy_score(pred_values, y_test)    
    lista_acc_score.append(acc)

    prec = precision_score(pred_values, y_test, average = 'weighted')
    lista_precision_score.append(prec)
    
    recall = recall_score(pred_values, y_test, average = 'weighted')
    lista_recall_score.append(recall)
    
    f1 = f1_score(pred_values, y_test, average = 'weighted')     
    lista_f1_score.append(f1)
    
media_conf_matrix = np.mean(lista_conf_matrix, axis=0)
media_acc_score = sum(lista_acc_score)/k
media_prec_score = sum(lista_precision_score)/k
media_recall_score = sum(lista_recall_score)/k
media_f1_score = sum(lista_f1_score)/k

print('Matriz de confusão:')
print(media_conf_matrix)
#print('Acurácia de cada iteração : {}'.format(acc_score))
print('Acurácia média: {}'.format(media_acc_score))
print('Precisão média: {}'.format(media_prec_score))
print('Recall médio  : {}'.format(media_recall_score))
print('F1 médio      : {}'.format(media_f1_score))     

Matriz de confusão:
[[374.2  23. ]
 [ 68.8 113.6]]
Acurácia média: 0.8416085998451551
Precisão média: 0.8672880266717173
Recall médio  : 0.8416085998451551
F1 médio      : 0.848250156886835


# Testes com dataset COVID19BR - Regressão Logística

In [8]:
X_ = tfidf_vectorizer_vectors

#Implementando a validação cruzada
 
k = 5
kf = KFold(n_splits=k, random_state=None)
model = LogisticRegression(solver= 'liblinear')

#lista das matrizes de confusão de cada iteração 
lista_conf_matrix = []
lista_acc_score = []
lista_precision_score = []
lista_recall_score = []
lista_f1_score = []
 
for train_index , test_index in kf.split(X):
    X_train , X_test = X_[train_index],X_[test_index]
    y_train , y_test = y[train_index],y[test_index]
     
    model.fit(X_train,y_train)
    pred_values = model.predict(X_test)
     
    #print(confusion_matrix(y_test, pred_values))
    conf_matrix = confusion_matrix(y_test, pred_values)
    lista_conf_matrix.append(conf_matrix)
    
    acc = accuracy_score(pred_values, y_test)    
    lista_acc_score.append(acc)

    prec = precision_score(pred_values, y_test, average = 'weighted')
    lista_precision_score.append(prec)
    
    recall = recall_score(pred_values, y_test, average = 'weighted')
    lista_recall_score.append(recall)
    
    f1 = f1_score(pred_values, y_test, average = 'weighted')     
    lista_f1_score.append(f1)
    
media_conf_matrix = np.mean(lista_conf_matrix, axis=0)
media_acc_score = sum(lista_acc_score)/k
media_prec_score = sum(lista_precision_score)/k
media_recall_score = sum(lista_recall_score)/k
media_f1_score = sum(lista_f1_score)/k

print('Matriz de confusão:')
print(media_conf_matrix)
#print('Acurácia de cada iteração : {}'.format(acc_score))
print('Acurácia média: {}'.format(media_acc_score))
print('Precisão média: {}'.format(media_prec_score))
print('Recall médio  : {}'.format(media_recall_score))
print('F1 médio      : {}'.format(media_f1_score))     

Matriz de confusão:
[[386.   11.2]
 [ 99.6  82.8]]
Acurácia média: 0.8088350902269074
Precisão média: 0.8882775799415125
Recall médio  : 0.8088350902269074
F1 médio      : 0.828811982023665


# Testes com dataset COVID19BR - GradientBoostingClassifier

In [9]:
X_ = tfidf_vectorizer_vectors

#Implementando a validação cruzada
 
k = 5
kf = KFold(n_splits=k, random_state=None)
model = GradientBoostingClassifier()

#lista das matrizes de confusão de cada iteração 
lista_conf_matrix = []
lista_acc_score = []
lista_precision_score = []
lista_recall_score = []
lista_f1_score = []
 
for train_index , test_index in kf.split(X):
    X_train , X_test = X_[train_index],X_[test_index]
    y_train , y_test = y[train_index],y[test_index]
     
    model.fit(X_train,y_train)
    pred_values = model.predict(X_test)
     
    #print(confusion_matrix(y_test, pred_values))
    conf_matrix = confusion_matrix(y_test, pred_values)
    lista_conf_matrix.append(conf_matrix)
    
    acc = accuracy_score(pred_values, y_test)    
    lista_acc_score.append(acc)

    prec = precision_score(pred_values, y_test, average = 'weighted')
    lista_precision_score.append(prec)
    
    recall = recall_score(pred_values, y_test, average = 'weighted')
    lista_recall_score.append(recall)
    
    f1 = f1_score(pred_values, y_test, average = 'weighted')     
    lista_f1_score.append(f1)
    
media_conf_matrix = np.mean(lista_conf_matrix, axis=0)
media_acc_score = sum(lista_acc_score)/k
media_prec_score = sum(lista_precision_score)/k
media_recall_score = sum(lista_recall_score)/k
media_f1_score = sum(lista_f1_score)/k

print('Matriz de confusão:')
print(media_conf_matrix)
#print('Acurácia de cada iteração : {}'.format(acc_score))
print('Acurácia média: {}'.format(media_acc_score))
print('Precisão média: {}'.format(media_prec_score))
print('Recall médio  : {}'.format(media_recall_score))
print('F1 médio      : {}'.format(media_f1_score))     

Matriz de confusão:
[[375.2  22. ]
 [ 82.  100.4]]
Acurácia média: 0.8205675659579537
Precisão média: 0.8611503355827816
Recall médio  : 0.8205675659579537
F1 médio      : 0.8313613559504693


# Testes com dataset COVID19BR - Árvore de decisão

In [10]:
X_ = tfidf_vectorizer_vectors

#Implementando a validação cruzada
 
k = 5
kf = KFold(n_splits=k, random_state=None)
model = tree.DecisionTreeClassifier()

#lista das matrizes de confusão de cada iteração 
lista_conf_matrix = []
lista_acc_score = []
lista_precision_score = []
lista_recall_score = []
lista_f1_score = []
 
for train_index , test_index in kf.split(X):
    X_train , X_test = X_[train_index],X_[test_index]
    y_train , y_test = y[train_index],y[test_index]
     
    model.fit(X_train,y_train)
    pred_values = model.predict(X_test)
     
    #print(confusion_matrix(y_test, pred_values))
    conf_matrix = confusion_matrix(y_test, pred_values)
    lista_conf_matrix.append(conf_matrix)
    
    acc = accuracy_score(pred_values, y_test)    
    lista_acc_score.append(acc)

    prec = precision_score(pred_values, y_test, average = 'weighted')
    lista_precision_score.append(prec)
    
    recall = recall_score(pred_values, y_test, average = 'weighted')
    lista_recall_score.append(recall)
    
    f1 = f1_score(pred_values, y_test, average = 'weighted')     
    lista_f1_score.append(f1)
    
media_conf_matrix = np.mean(lista_conf_matrix, axis=0)
media_acc_score = sum(lista_acc_score)/k
media_prec_score = sum(lista_precision_score)/k
media_recall_score = sum(lista_recall_score)/k
media_f1_score = sum(lista_f1_score)/k

print('Matriz de confusão:')
print(media_conf_matrix)
#print('Acurácia de cada iteração : {}'.format(acc_score))
print('Acurácia média: {}'.format(media_acc_score))
print('Precisão média: {}'.format(media_prec_score))
print('Recall médio  : {}'.format(media_recall_score))
print('F1 médio      : {}'.format(media_f1_score))     

Matriz de confusão:
[[344.2  53. ]
 [ 67.4 115. ]]
Acurácia média: 0.7922738371746769
Precisão média: 0.7977454785443405
Recall médio  : 0.7922738371746769
F1 médio      : 0.7939070671298609


# Testes com o dataset COVID19BR - Rede Neural MLP

In [11]:
X_ = tfidf_vectorizer_vectors

#Implementando a validação cruzada
 
k = 5
kf = KFold(n_splits=k, random_state=None)
model = MLPClassifier()

#lista das matrizes de confusão de cada iteração 
lista_conf_matrix = []
lista_acc_score = []
lista_precision_score = []
lista_recall_score = []
lista_f1_score = []
 
for train_index , test_index in kf.split(X):
    X_train , X_test = X_[train_index],X_[test_index]
    y_train , y_test = y[train_index],y[test_index]
     
    model.fit(X_train,y_train)
    pred_values = model.predict(X_test)
     
    #print(confusion_matrix(y_test, pred_values))
    conf_matrix = confusion_matrix(y_test, pred_values)
    lista_conf_matrix.append(conf_matrix)
    
    acc = accuracy_score(pred_values, y_test)    
    lista_acc_score.append(acc)

    prec = precision_score(pred_values, y_test, average = 'weighted')
    lista_precision_score.append(prec)
    
    recall = recall_score(pred_values, y_test, average = 'weighted')
    lista_recall_score.append(recall)
    
    f1 = f1_score(pred_values, y_test, average = 'weighted')     
    lista_f1_score.append(f1)
    
media_conf_matrix = np.mean(lista_conf_matrix, axis=0)
media_acc_score = sum(lista_acc_score)/k
media_prec_score = sum(lista_precision_score)/k
media_recall_score = sum(lista_recall_score)/k
media_f1_score = sum(lista_f1_score)/k

print('Matriz de confusão:')
print(media_conf_matrix)
#print('Acurácia de cada iteração : {}'.format(acc_score))
print('Acurácia média: {}'.format(media_acc_score))
print('Precisão média: {}'.format(media_prec_score))
print('Recall médio  : {}'.format(media_recall_score))
print('F1 médio      : {}'.format(media_f1_score))     

Matriz de confusão:
[[364.2  33. ]
 [ 56.6 125.8]]
Acurácia média: 0.8453999166219999
Precisão média: 0.8559779125277496
Recall médio  : 0.8453999166219999
F1 médio      : 0.848161722211166


# Início do tutorial encontrado em:
https://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/#.Y1Ccl3bMJPY

Idéia inicial do KFold: # https://www.askpython.com/python/examples/k-fold-cross-validation

In [ ]:
#instancia CountVectorizer() 
cv=CountVectorizer() 
# estes passos geram 
# this steps generates word counts for the words in your docs 
word_count_vector=cv.fit_transform(X) # o X são as postagens do meu arquivo

In [ ]:
word_count_vector.shape

In [ ]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(word_count_vector)

In [ ]:
# print idf values 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"]) 
# sort ascending 
df_idf.sort_values(by=['idf_weights'])

In [ ]:
# count matrix 
count_vector=cv.transform(X) 
# tf-idf scores 
tf_idf_vector=tfidf_transformer.transform(count_vector)

In [ ]:
feature_names = cv.get_feature_names() 
#get tfidf vector for first document 
first_document_vector=tf_idf_vector[0] 
#print the scores 
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"]) 
df.sort_values(by=["tfidf"],ascending=False)

In [ ]:
#from sklearn.feature_extraction.text import TfidfVectorizer 
# settings that you use for count vectorizer will go here 
tfidf_vectorizer=TfidfVectorizer(use_idf=True) 
# just send in all your docs here 
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(X)

In [ ]:
# get the first vector out (for the first document) 
first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[0] 
# place tf-idf values in a pandas data frame 
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), 
                  index=tfidf_vectorizer.get_feature_names(), 
                  columns=["tfidf"])
df.sort_values(by=["tfidf"], ascending=False)

In [ ]:
#Pode ser resumido nisto: 
tfidf_vectorizer=TfidfVectorizer(use_idf=True)
# just send in all your docs here
fitted_vectorizer=tfidf_vectorizer.fit(X)
tfidf_vectorizer_vectors=fitted_vectorizer.transform(X)

# Fim do tutorial

In [ ]:
#Verificando o resultado após a limpeza de texto
 
contador = 0
for i in X:
    contador += 1
    print(i)
    if contador > 5:
        break

In [ ]:
#Verificando o resultado após a vetorização TF-IDF
 
contador = 0
for i in X_:
    contador += 1
    print(i)
    if contador > 1:
        break